# Hard and soft constraints

### Hard constraints
These are constraints that must be pursued during the search and without meeting these constraints, the solution is not true. These constraints are:
- One teacher cannot give two courses at the same time
- A set of students cannot have two classes at the same time
- A teacher cannot teach more than maximum allotted hours.
- Classes Cannot be on off days (Saturday and Sunday)
- Maximum number of hours allotted for a day cannot be exceeded.

### Soft constraints
These constraints that are not mandatory for a solution must be pursued, but the quality of the courses schedule is decided by following these constraints
- In the timetable of teacher, no more than two lectures a day, each one 1 hours and 30 minutes, breaks are preferred 30 minutes or more.
- In the students' timetable, we have to put three lectures a day at maximum with 30 minutes breaks between lectures so that they have more time for self-study at home
- We must ensure that a group of students does not have to attend school only to attend one class, there has to be a maximum of three classes in a day and a minimum of two classes so that it will be worth it to come to college on a particular day

# Initialization

In [55]:
def initialize(students, courses, day_times, prof_input):
    initialized_data = {}
    initialized_data["stgs"] = students
    initialized_data["courses"] = courses
    initialized_data["days"] = [i for i in range(day_times["day"])]
    initialized_data["periods"] = [i for i in range(day_times["period"])]
    initialized_data["profs"] = prof_input
    return initialized_data

In [85]:
# Fitness function
def fitness_function(matrix, data):
    stgs = [ key for key in data["stgs"].keys()]
    courses = [ key for key in data["courses"].keys()]
    profs = [key for key in data["profs"].keys()]
    stg_checker = {}
    course_checker = {}
    prof_checker = {}
    for stg in stgs:
        stg_checker[stg] = []
    for prof in profs:
        prof_checker[prof] = []
    for course in courses:
        course_checker[course] = 0

    # Check for hard constraints
    hard_penalty_counter = 0
    for chromosome in matrix:
        stg = chromosome["stg"]
        course = chromosome["course"]
        prof = chromosome["prof"]
        day = chromosome["day"]
        period = chromosome["period"]
        course_checker[course] += 1
        if course not in data["stgs"][stg]["course_list"]:
            # student group doesn't learn the course
            hard_penalty_counter += 1
        if prof not in data["courses"][course]["prof_list"]:
            # prof doesn't teach
            hard_penalty_counter += 1
        timeslot = [day, period]
        if timeslot not in stg_checker[stg]:
            stg_checker[stg].append(timeslot)
        else:
            # conflict
            hard_penalty_counter += 1
        if timeslot not in prof_checker[prof]:
            prof_checker[prof].append(timeslot)
        else:
            # conflict
            hard_penalty_counter += 1
        if timeslot not in data["profs"][prof]["availability_list"]:
            # prof not available
            # print("unavailable")
            hard_penalty_counter += 1
    for course_key in course_checker.keys():
        if course_checker[course_key] != data["courses"][course]["hour"]:
            # Number of weekly hours not equal
            # print("weekly hour limit")
            hard_penalty_counter += 1
    
    # Check for soft constraints
    soft_penalty_counter = 0

    # print("stg", stg_checker)
    # print("prof", prof_checker)
    # print("course_checker", course_checker)
    # print("hard_penalty_counter", hard_penalty_counter)
    result = 1/(1+(soft_penalty_counter + hard_penalty_counter))
    return result

In [59]:
# Code for initialization based on input
# Number of population = number student group weekly hours
student_input = {0: {"hour": 3, "course_list": [0, 1]}, 1: {"hour": 3, "course_list": [1]}}
course_input = {0: {"hour": 3, "prof_list": [0, 1]}, 1: {"hour": 3, "prof_list": [1]}}
day_time_availability = {"day": 5, "period": 8}
# [[day, period]]
prof_input = {0: {"availability_list": [[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7],[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7],[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7] ]},
1: {"availability_list": [[3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7]]}}
initialized_data = initialize(student_input, course_input, day_time_availability, prof_input)
print(initialized_data)

{'stgs': {0: {'hour': 3, 'course_list': [0, 1]}, 1: {'hour': 3, 'course_list': [1]}}, 'courses': {0: {'hour': 3, 'prof_list': [0, 1]}, 1: {'hour': 3, 'prof_list': [1]}}, 'days': [0, 1, 2, 3, 4], 'periods': [0, 1, 2, 3, 4, 5, 6, 7], 'profs': {0: {'availability_list': [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7]]}, 1: {'availability_list': [[3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7]]}}}


In [39]:
stgs = [ key for key in initialized_data["stgs"].keys()]
courses = [ key for key in initialized_data["courses"].keys()]
profs = [key for key in initialized_data["profs"].keys()]
days = initialized_data["days"]
periods = initialized_data["periods"]

In [74]:
# Initial individual with fitness value = 1
# one of possible solutions: 
chromosome_0 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[0]}
chromosome_1 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[1]}
chromosome_2 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[2]}
chromosome_3 = {"stg": stgs[1], "course": courses[1], "prof": profs[1], "day": days[3], "period": periods[0]}
chromosome_4 = {"stg": stgs[1], "course": courses[1], "prof": profs[1], "day": days[3], "period": periods[1]}
chromosome_5 = {"stg": stgs[1], "course": courses[1], "prof": profs[1], "day": days[3], "period": periods[2]}
individual_0 = [chromosome_0, chromosome_1, chromosome_2, chromosome_3, chromosome_4, chromosome_5]
print(fitness_function(individual_0, initialized_data))

1.0


In [65]:
import numpy as np
from numpy import random
# Automating population generation
def generate_population(n, input_data):
    stgs = [ key for key in input_data["stgs"].keys()]
    courses = [ key for key in input_data["courses"].keys()]
    profs = [key for key in input_data["profs"].keys()]
    days = input_data["days"]
    periods = input_data["periods"]

    population = []
    for _ in range(n):
        cur_individual = []
        for stg in stgs:
            for _ in range(input_data["stgs"][stg]["hour"]):
                course = random.randint(len(courses))
                prof = random.randint(len(profs))
                day = random.randint(len(days))
                period = random.randint(len(periods))
                cur_chromosome = {"stg": stg, "course": course, "prof": prof,
                "day": day, "period": period}
                cur_individual.append(cur_chromosome)
        population.append(cur_individual)
    return population

In [170]:
population = generate_population(100000, initialized_data)
# print(population)
idx_fitness_value = []
for idx in range(len(population)):
    fitness_value = fitness_function(population[idx], initialized_data)
    idx_fitness_value.append([idx, fitness_value])
sorted_idx_fitness = sorted(idx_fitness_value,key=lambda x: x[1], reverse=True)
print(sorted_idx_fitness[:10])

[[51838, 1.0], [63133, 1.0], [68539, 1.0], [656, 0.5], [3640, 0.5], [6264, 0.5], [6934, 0.5], [6992, 0.5], [10954, 0.5], [36428, 0.5]]


In [171]:
print(population[656])

[{'stg': 0, 'course': 0, 'prof': 0, 'day': 2, 'period': 1}, {'stg': 0, 'course': 0, 'prof': 1, 'day': 4, 'period': 7}, {'stg': 0, 'course': 0, 'prof': 0, 'day': 2, 'period': 3}, {'stg': 1, 'course': 1, 'prof': 1, 'day': 3, 'period': 4}, {'stg': 1, 'course': 1, 'prof': 1, 'day': 3, 'period': 7}, {'stg': 1, 'course': 1, 'prof': 1, 'day': 4, 'period': 4}]


# Evolutionary steps

In [12]:
# Initial population, consist of a list of chromosomes
time_table_0 = stg_0 + course_0 + prof_0 + day_0 + time_0
time_table_1 = stg_0 + course_1 + prof_1 + day_0 + time_1
time_table_2 = stg_0 + course_2 + prof_2 + day_0 + time_2

In [13]:
print(time_table_0)
print(time_table_1)
print(time_table_2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
